# Tratamento de dados do World Bank

Este notebook Jupyter realiza o tratamento de um dataset do Banco Mundial, transformando-o de um formato largo para um formato longo e, em seguida, pivotando-o para um formato largo com indicadores como colunas.

## Importações das bibliotecas

As seguintes bibliotecas são importadas para a manipulação de dados:

-   `numpy` para operações numéricas.
-   `pandas` para manipulação de DataFrames.
-   `re` para Regex.

In [2]:
import numpy as np
import pandas as pd
import re

## Leitura e Limpeza Inicial dos Dados

1.  **Leitura do Dataset:**
    O dataset bruto é lido do arquivo `worldbank_data.csv` usando `pd.read_csv()`.
2.  **Remoção da Coluna 'Series Code':**
    A coluna 'Series Code' é removida, pois não é necessária para a análise.
3.  **Extração dos Anos:**
    Os anos são extraídos dos nomes das colunas usando expressões regulares.
4.  **Renomeação das Colunas:**
    As colunas são renomeadas para facilitar a manipulação e o entendimento.


In [3]:
raw_df = pd.read_csv('datasets/worldbank_data.csv')
raw_df.drop(columns='Series Code', inplace=True)
year_columns = [re.search(r'\d{4}', col).group(0) for col in raw_df.columns[3:]]
id_columns = ["country_name", "country_code", "indicator_name"]
raw_df.columns= id_columns + year_columns

## Transformação dos Dados para o Formato Longo

1.  **Melt do DataFrame:**
    O DataFrame é transformado para o formato longo usando `pd.melt()`, com as colunas de identificação como `id_vars`.
2.  **Remoção de Duplicatas:**
    As linhas duplicadas são removidas para garantir a integridade dos dados.

In [4]:
df_melted = pd.melt(
    raw_df,
    id_vars=id_columns,
    var_name='year',
    value_name='indicator_value'
)

df_melted = df_melted.drop_duplicates(
    subset=['country_name', 'country_code', 'year', 'indicator_name']
)

## Pivotagem dos Dados para o Formato Largo

1.  **Pivot do DataFrame:**
    O DataFrame é pivotado para o formato largo usando `pd.pivot()`, com os indicadores como colunas.
2.  **Substituição de NaN por None:**
    Os valores NaN são substituídos por None para evitar problemas com bancos de dados.

In [5]:
df_pivoted = df_melted.pivot(
    index=['country_name', 'country_code', 'year'],
    columns='indicator_name',
    values='indicator_value'
).reset_index().rename_axis(None, axis=1)

df_pivoted = df_pivoted.replace({np.nan: None})


## Salvamento dos Dados Tratados

O DataFrame tratado é salvo em um arquivo CSV `treated_dataset.csv` usando `pandas.to_csv()`.

In [6]:
df_pivoted.to_csv('treated_dataset.csv', index=False)